In [ ]:
import sys
import pandas
sys.path.append("..") 
sys.path.append("../src") 
from src import dataFiles, dataMangling, dataPlotting, dataTable

In [ ]:
dm = dataMangling.dataMangled()
ts, bnn = dataFiles.data(withSynthetic=True)
dates = dataMangling.dates_list(ts)
ts_BuLa, Bundeslaender = dataMangling.join_tables_for_and_aggregate_Bundeslaender(ts, bnn)

In [ ]:
"\n".join(ts[ts.ADMIN.isna()]["AGS"].tolist())

In [ ]:
Bundeslaender

In [ ]:
fed = dataMangling.get_BuLa(Bundeslaender, "Hessen", dm.datacolumns)
daily, cumulative, title, filename, population = fed.daily, fed.cumulative, fed.title, fed.filename, fed.population

In [ ]:
center = dataMangling.temporal_center(daily)[0]
center

In [ ]:
dataMangling.temporal_center(fed.daily)

In [ ]:
Bundeslaender.index.tolist()

In [ ]:
Bundeslaender["centerday"] = [dataMangling.get_BuLa(Bundeslaender, name, dm.datacolumns).center
                                  for name in Bundeslaender.index.tolist() ]

In [ ]:
Bundeslaender

In [ ]:
Bundeslaender.sort_values("centerday",inplace=True)

In [ ]:
Bundeslaender

In [ ]:
Bundeslaender.loc['Deutschland'] = Bundeslaender.sum().values.tolist()

In [ ]:
ts

In [ ]:
ts.drop(ts[ts.ADMIN.isna()].index, inplace=True)

ts

In [ ]:
import io, sys

import pandas
import requests

sys.path.append("..") 
sys.path.append("../src") 

from dataFiles import OPENDATASOFT_URL01, OPENDATASOFT_URL02, DISTANCES_PATH 

def download_kreise_locations(url1=OPENDATASOFT_URL01, url2=OPENDATASOFT_URL02):
    print ("Downloading large table with Kreise locations. For infos see")
    print (url1)
    print ("Patience please ...", end=" ")
    s=requests.get(url2).content
    LKG=pandas.read_csv(io.StringIO(s.decode('utf-8')), sep=';') # error_bad_lines=False)
    # immediately drop non-Kreis row(s)
    LKG.dropna(subset=['Cca 2'],inplace=True)
    # turn AGS into integer:
    LKG["Cca 2"]=LKG["Cca 2"].astype(int)
    print ("Done downloading.")
    return LKG
LKG=download_kreise_locations()

In [ ]:
LKG

In [ ]:
# accelerate lookup
AGS_to_geopoint=dict(LKG[["Cca 2","Geo Point"]].set_index("Cca 2").to_dict('series')["Geo Point"])
AGS_to_geopoint

In [ ]:
import geopy.distance

def geo_distance(AGS_to_geopoint, AGS1, AGS2):
    c1 = list(map(float,AGS_to_geopoint[AGS1].split(",")))
    c2 = list(map(float,AGS_to_geopoint[AGS2].split(",")))
    # print (c1, c2 )
    return geopy.distance.geodesic(c1, c2).kilometers

print (geo_distance(AGS_to_geopoint, 1001, 1002))

In [ ]:
def make_distances_table(AGS_to_geopoint, all_AGS, filename=DISTANCES_PATH):
    headers = ['AGS1','AGS2','km']
    counter,countermax=0, len(all_AGS)*(len(all_AGS)-1)
    distances=pandas.DataFrame([], columns = headers)
    for AGS1 in all_AGS:
        for AGS2 in all_AGS:
            if AGS1>=AGS2:
                continue
            geodist = geo_distance(AGS_to_geopoint, AGS1, AGS2)
            row=pandas.Series([AGS1, AGS2, geodist], index=headers)
            distances=distances.append(row, ignore_index=True)
            row=pandas.Series([AGS2, AGS1, geodist], index=headers)
            distances=distances.append(row, ignore_index=True)
            counter+=2
            if not counter%100:
                print (counter, "/", countermax)

    print ("Done calculating distances. Len(table)=%d" % len(distances))

    distances["AGS1"]=distances["AGS1"].astype(int)
    distances["AGS2"]=distances["AGS2"].astype(int)
        
    print ("Distances are distributed like this:")
    print(distances["km"].describe())
    
    distances.sort_values("km", inplace=True)
    print("Done sorting.")
    
    distances.to_csv(filename)
    print ("Saved to", filename)
    return distances, filename

all_AGS=LKG["Cca 2"].dropna().astype(int).tolist()
# all_AGS=all_AGS[:20] # reduce number for dev'ing
print(all_AGS)
print()

distances, filename = make_distances_table(AGS_to_geopoint, all_AGS)
distances

In [ ]:
distances

In [ ]:
(distances[(distances.km<150) & (distances.AGS1==9371)])

In [ ]:
import sys
sys.path.append("..") 
sys.path.append("../src") 
import dataFiles, dataMangling, dataPlotting, districtDistances
distances = districtDistances.load_distances()
print (districtDistances.nearby(distances, 1001, 50))


In [ ]:
distances[(distances.km<=50)]

In [ ]:
ts, bnn = dataFiles.data(withSynthetic=True)

In [ ]:
bnn.BEZ.unique()

In [ ]:
LKG

In [ ]:
def compare_risklayer_with_opendatasoft(bnn):
    filename = dataFiles.OPENDATASOFT_PATH
    LKG = districtDistances.load_kreise_locations(filename)
    ODS = set(LKG["Cca 2"].tolist())
    ts, bnn = dataFiles.data(withSynthetic=True)
    RSL=set(bnn["AGS"].values.tolist())
    
    diff1 = list(ODS-RSL)
    names1=([LKG["Name 2"][LKG["Cca 2"]==AGS].tolist()[0] for AGS in diff1])
    print ("ODS-RSL: %s = %s"% (diff1, names1))
    diff2 = list(RSL-ODS)
    names2=([bnn["GEN"][bnn["AGS"]==AGS].tolist()[0] for AGS in diff2])
    print ("RSL-ODS: %s = %s"% (diff2, names2))
compare_risklayer_with_opendatasoft(bnn)

In [ ]:
i=LKG[LKG["Cca 2"]==3152].index.tolist()
if i:
    LKG.at[i, "Cca 2"] = 3159

In [ ]:
LKG.loc[i]

In [ ]:
i=LKG[LKG["Cca 2"]==3156].index.tolist()
if i:
    LKG.drop(index=i, inplace=True)

In [ ]:
LKG[(LKG["Cca 2"]> 3150) & (LKG["Cca 2"]< 3161)]

In [ ]:
a=[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 9.0]
list(map(int, a))

In [ ]:
import sys
sys.path.append("..") 
sys.path.append("../src") 
import dataFiles, dataMangling, dataPlotting, districtDistances,dataTable
ts, bnn, ts_sorted, Bundeslaender_sorted, dates, datacolumns = dataMangling.dataMangled()
distances = districtDistances.load_distances()
cmap = dataTable.colormap()


In [ ]:
Bundeslaender_sorted.index.tolist()


In [ ]:
DE=Bundeslaender_sorted.drop(["Deutschland", "Dummyland"]).sum()
DE[datacolumns].astype(int).tolist()

In [ ]:
DE = Bundeslaender_sorted.sum()

In [ ]:
DE["Population"]

In [ ]:
import sys
import pandas
sys.path.append("..") 
sys.path.append("../src") 
import dataFiles, dataMangling, dataPlotting, districtDistances,dataTable
dataFiles.TS_NEWEST

In [ ]:
ts=pandas.read_csv(dataFiles.TS_NEWEST, encoding='cp1252') # encoding='utf-8')
ts[ts["AGS"]=="05370"]

In [ ]:
ts["28.04.2020"]=(ts["29.04.2020"]+ts["27.04.2020"])/2

In [ ]:
ts[ts["AGS"]=="05370"]

In [ ]:
import sys
sys.path.append("..") 
sys.path.append("../src") 
import dataFiles, dataMangling, dataPlotting, districtDistances,dataTable
ts, bnn, ts_sorted, Bundeslaender_sorted, dates, datacolumns = dataMangling.dataMangled()

In [ ]:

BLs=sorted(Bundeslaender_sorted.index.tolist())
BLs = [BL for BL in BLs if BL not in ("Dummyland", "Deutschland")]

global page
page=""
c=0

class page(object):
    page=""
    def a(self, t):
        self.page+=t+"\n"
       
p=page()
p.a("<table>")
for i in range(4):
    p.a("<tr>")
    for j in range(4):
        print(c, BLs[c])
        imgprop='src="https://covh.github.io/cov19de/pics/bundesland_%s.png" alt="bundesland_%s.png"'%(BLs[c],BLs[c])
        p.a('<td><a href="%s.html"><img %s width="458" height="268"></a></td>' % (BLs[c], imgprop))
        c+=1
    p.a("</tr>")
p.a("</table>")
print (p.page)

In [ ]:
ts_sorted

In [ ]:
def AGS_to_cumulative(ts_rich, datacolumns, AGS):
    """
    now also accepts tables with additional columns
    """
    AGS = ("00000%s"%AGS)[-5:]
    row = ts.loc[ts['AGS'] == AGS]
    print 
    return row[datacolumns].values[0].tolist()
print (AGS_to_cumulative(ts_rich=ts_sorted, datacolumns=datacolumns, AGS=1001))

In [ ]:
import pandas, numpy

def AGS_to_cumulative(ts_rich, datacolumns, AGS):
    """
    now also accepts tables with additional columns
    because positive selection by datacolumns (not negative by droppping columns)  
    """
    AGS = ("00000%s"%AGS)[-5:]
    row = ts.loc[ts['AGS'] == AGS]
    print 
    return row[datacolumns].values[0].tolist()


def AGS_to_daily(ts_rich, datacolumns, AGS):
    """
    now also accepts tables with additional columns
    because positive selection by datacolumns (not negative by droppping columns)  
    """
    cum = pandas.Series ( AGS_to_cumulative(ts_rich, datacolumns, AGS) ) 
    diff = cum.diff()
    return diff.values.tolist()

AGS=5370
print (AGS_to_cumulative(ts_rich=ts_sorted, datacolumns=datacolumns, AGS=AGS))
print (AGS_to_daily(ts_rich=ts_sorted, datacolumns=datacolumns, AGS=AGS))

In [ ]:
AGS=1001
AGS=5370
cumulative=AGS_to_cumulative(ts_rich=ts_sorted, datacolumns=datacolumns, AGS=AGS)

def cumulative_smoothed_last_week_incidence(cumulative, windowsize=7, daysBack=7):
    averaged=pandas.DataFrame(cumulative).rolling(window=windowsize, center=False).mean()[0].values.tolist()
    return averaged[-1]-averaged[-daysBack]
cumulative_smoothed_last_week_incidence(cumulative)

def cumulative_today_minus_last_week_smoothed(cumulative, windowsize=7, daysBack=7):
    averaged=pandas.DataFrame(cumulative).rolling(window=windowsize, center=True).mean()[0].values.tolist()
    return cumulative[-1] - averaged[-daysBack]
cumulative_today_minus_last_week_smoothed(cumulative)

In [ ]:
ts_rich = ts_sorted
ts_rich

In [ ]:
ts_rich.loc[7135][datacolumns].tolist()

In [ ]:
Bundeslaender_sorted

In [ ]:
def multiDayIncidence(cumulative, daysBack=7):
    """
    unaveraged, will probably work best for daysBack=7 or 14 or 21 
    """
    return cumulative[-1] - cumulative[-daysBack-1]


def add_incidence_columns(ts_rich):
    
    for days in (7, 14):
        ts_rich["incidence_last%days" % days] = [ multiDayIncidence( AGS_to_cumulative(ts_rich, datacolumns, AGS) , days)
                                                  for AGS in ts_rich.index.values.tolist() ]
    return ts_sorted


def add_incidence_columns_Bundeslaender(Bundeslaender):
    
    for days in (7, 14):
        colname = "incidence_last%ddays" % days
        Bundeslaender[colname] = [ multiDayIncidence(  get_BuLa(Bundeslaender, name, dm.datacolumns).cumulative , days)
                                  for name in Bundeslaender.index.tolist() ]

    return Bundeslaender


add_incidence_columns(ts_sorted)
add_incidence_columns_Bundeslaender(Bundeslaender_sorted)
    

In [ ]:
AGS_to_cumulative(ts_sorted, datacolumns, AGS)

In [ ]:
from dataMangling import get_BuLa
add_incidence_columns_Bundeslaender(Bundeslaender_sorted)

In [ ]:
[ multiDayIncidence( AGS_to_cumulative(ts_sorted, datacolumns, AGS) , days)
                                                  for AGS in ts_sorted["AGS"].tolist() ]

In [ ]:

multiDayIncidence( AGS_to_cumulative(ts_rich, datacolumns, AGS) )

In [ ]:
ts_rich.index.values.tolist()

In [ ]:
def add_incidence_columns(ts_rich):
    
    for days in (7, 14):
        ts_rich["incidence_last%days" % days] = [ multiDayIncidence( AGS_to_cumulative(ts_rich, datacolumns, AGS) , days)
                                                  for AGS in ts_rich.index.values.tolist() ]
    return ts_sorted

In [ ]:
import sys
import numpy,matplotlib,pandas
sys.path.append("..") 
sys.path.append("../src") 
import dataFiles, dataMangling, dataPlotting, districtDistances,dataTable
ts, bnn, ts_sorted, Bundeslaender_sorted, dates, datacolumns = dataMangling.dataMangled()

In [ ]:
Bundeslaender_sorted
ts_sorted

In [ ]:
AGS=9377
AGS=9479
AGS=16076
cumulative=ts_sorted.loc[AGS][datacolumns].tolist()
daily=ts_sorted.loc[AGS][datacolumns].diff().tolist()
daily_SMA=pandas.DataFrame(daily).rolling(window=7, center=True).mean()[0].values.tolist()
daily_SMA
list(zip(cumulative,daily, daily_SMA))

In [ ]:
def Reff_4_4(daily,i):
    if i<7:
        return numpy.nan
    d=daily
    denominator = d[i-4]+d[i-5]+d[i-6]+d[i-7]
    result = (d[i]+d[i-1]+d[i-2]+d[i-3]) / denominator if denominator else numpy.nan
    return result 

def Reff_4_7(daily,i):
    if i<10:
        return numpy.nan
    d=daily
    denominator = d[i-4]+d[i-5]+d[i-6]+d[i-7]+d[i-8]+d[i-9]+d[i-10]
    result = (d[i]+d[i-1]+d[i-2]+d[i-3]+d[i-4]+d[i-5]+d[i-6]) / denominator if denominator else numpy.nan
    return result 

R1=[Reff_4_4(daily, i) for i, _ in enumerate(daily)]
R2=[Reff_4_7(daily, i) for i, _ in enumerate(daily)]
R3=[Reff_4_4(daily_SMA, i) for i, _ in enumerate(daily_SMA)]

R1
matplotlib.pyplot.plot(R1)

In [ ]:
Bundeslaender_sorted